In [93]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot') 

%matplotlib inline

In [252]:
column_labels = ['rsid', 'chromosome', 'position', 'genotype']
dtypes = {'rsid': object, 'chromosome': object, 'position': int, 'genotype': object}

In [55]:
file_name_no_header = '/Volumes/DSI/opensnp_datadump.current/user757_file359_yearofbirth_unknown_sex_unknown.23andme.txt'

df1 = pd.read_table(file_name, delimiter="\t", header=None, names=column_labels)

In [56]:
df1.head()

,rsid,chromosome,position,genotype
0,rs2131925,0,1,TT
1,rs2815752,0,1,AA
2,rs10924081,0,1,AA
3,rs984222,0,1,CG
4,rs7522380,0,1,AG


In [57]:
file_name_nw = '/Volumes/DSI/opensnp_datadump.current/user4435_file3040_yearofbirth_unknown_sex_unknown.23andme.txt'
df2 = pd.read_table(file_name_nw, delimiter="\t", header=None, 
                   names=column_labels, skip_blank_lines=True, comment='#',
                   dtype={'rsid': object}
                  )

In [59]:
df2.head()

,rsid,chromosome,position,genotype
0,rs4897988,1,121484193,TA
1,rs9701907,1,121484197,AC
2,rs9658904,1,121484236,TC
3,rs4127590,1,121484261,AG
4,rs4788327,1,121484283,TA


In [68]:
pd.merge(df1, df2, how='inner', on=column_labels)

,rsid,chromosome,position,genotype


In [36]:
file_name_large = '/Volumes/DSI/opensnp_datadump.current/user672_file311_yearofbirth_1937_sex_XX.23andme.txt'

df = pd.read_table(file_name_large, delimiter="\t", 
                   header=None, names=column_labels, 
                   dtype=dtypes,
                   skip_blank_lines=True, comment='#')

In [38]:
df.shape

(960613, 4)

In [66]:
def open_file(filepath):
    df = pd.read_table(filepath, delimiter="\t", 
                   header=None, names=column_labels, 
                   dtype=dtypes,
                   skip_blank_lines=True, comment='#')
    return df

In [274]:
metadata = pd.read_csv('./phenotypes.csv', dtype=object, index_col='id')

df_model = pd.DataFrame(index=metadata.index)
# df_model['user_id'] = metadata.index
# df_model = df_model.set_index('user_id')

In [300]:
# metadata.loc[1]['variation_146_value']

In [395]:
df_model['migraine'] = metadata['variation_146_value']
df_model['migraine'] = df_model['migraine'].fillna(0)
df_model['migraine'] = df_model['migraine'].apply(lambda x: 1 if (str(x).lower() != 'no' and x != 0) else 0)

migraine_users_id = sorted(df_model[df_model['migraine'] == 1].index)
# migraine_users_id

In [396]:
df_model['ADHD'] = metadata['variation_29_value']
df_model['ADHD'] = df_model['ADHD'].fillna(0)
df_model['ADHD'] = df_model['ADHD'].apply(lambda x: 1 if (str(x).lower() != 'no' and x != 0 and x != 'False') else 0)

adhd_users_id = sorted(df_model[df_model['ADHD'] == 1].index)
# adhd_users_id

In [273]:
metadata.index == 2307

array([False, False, False, ..., False, False, False], dtype=bool)

In [397]:
df_model['IBS'] = metadata['variation_173_value']
df_model['IBS'] = df_model['IBS'].fillna(0)
df_model['IBS'] = df_model['IBS'].apply(lambda x: 1 if (str(x).lower() != 'no' and x != 0) else 0)

ibs_users_id = sorted(df_model[df_model['IBS'] == 1].index)
# ibs_users_id

In [398]:
df_model['HighBP'] = metadata['variation_33_value']
df_model['HighBP'] = df_model['HighBP'].fillna(0)
df_model['HighBP'] = df_model['HighBP'].apply(lambda x: 1 if 
          x in ['Prehypertension', 'Hypertension', 'Hypertenisve',
                 'Hypertensive w/ extreme orthostatic variations',
                 'Had hypertension, now normal.',
                 'Orthostatically variable (postural changes)',
                 'Prehypertension, low pulse',
                 'Hypertensive w/ Extreme Orthostatic Variations',
                 'hypertenisve', 'prehypertension'] else 0)

high_bp_users_id = sorted(df_model[df_model['HighBP'] == 1].index)
# high_bp_users_id

In [399]:
df_model['CFS'] = metadata['variation_66_value']
df_model['CFS'] = df_model['CFS'].fillna(0)
df_model['CFS'] = df_model['CFS'].apply(lambda x: 1 
        if (str(x).lower() == 'yes' 
            or x == 'After seeing a sleep doctor, i can happily say no longer' 
            or x == 'After seeing a sleep doctor, I can happily say No Longer')
        else 0)

cfs_users_id = sorted(df_model[df_model['CFS'] == 1].index)
# cfs_users_id

In [400]:
df_model['CF'] = metadata['variation_116_value']
df_model['CF'] = df_model['CF'].fillna(0)

def cf(value):
    print(value)
    print(str(value).lower() not in ['no', 'i dont know', '0'])
df_model['CF'] = df_model['CF'].apply(lambda x: 1 
        if str(x).lower() not in ['no', 'i dont know', '0']
        else 0)

cf_users_id = sorted(df_model[df_model['CF'] == 1].index)
cf_users_id

[763, 2385, 5397]

In [153]:
def unique(a):
    """ return the list with duplicate elements removed """
    return list(set(a))

def intersect(a, b):
    """ return the intersection of two lists """
    return sorted(list(set(a) & set(b)))

def union(a, b):
    """ return the union of two lists """
    return list(set(a) | set(b))


In [310]:
intersect(high_bp_users_id, migraine_users_id)

[1269, 2083, 2288, 2385, 3046, 3532, 4428, 5098, 5917]

In [213]:
intersect(intersect(intersect(adhd_users_id, migraine_users_id), high_bp_users_id), cfs_users_id)

[]

http://www.beyondmthfr.com/a-genetic-cause-of-pain-and-anxiety-comt-mao-and-mthfr/
* COMT rs6269 ## G/AA
* COMT_-61_P199P rs769224 ## A/GG
* COMT_H62H rs4633 ## T/TT
* COMT_V158M rs4680 ## A/AA
* MAO_A_R297R rs6323 ## T/T
* MTHFR_A1298C rs1801131 ## G/GT
* MTHFR_C677T rs1801133 ## A/AG

* rs6269, rs4633, rs4818 and (rs4680 or Val158Met)

* Rs4818 and rs4633 of COMT gene

** The work identified a common genetic variant rs13361160 on chromosome 5p15.2, associated with CWP, located upstream of the chaperonin-containing-TCP1-complex-5 gene (CCT5) and downstream of FAM173B. Although this large genetic study targeted CWP, thus not specifically FM, the work identified promising genetic targets for study of the pathogenesis of FM.

** One mutation commonly studied in the context of FM pathophysiology is the Val158Met polymorphism (SNP rs4680). This SNP is associated with a valine (Val GTG) to methionine (Met ATG) transition at codon 158 [47]. This Met substitution reduces the activity of the COMT enzyme. In fact, the H/H (GG; Val-158-Val) genotype generates a fully effective enzyme, but the H/L (AG; Met-158-Val) genotype an intermediate-activity enzyme. The L/L (AA; Met-158-Met) genotype produces a defective enzyme, which impairs the clearance of catecholamines from the CNS. 

** Other COMT gene polymorphisms, including the SNPs rs6269, rs4633, and rs4818, have also been evaluated in FM patients [49,50,53]. In a Korean population, we found that rs4818 and rs4633 may be associated with FM risk and pain sensitivity [56]. However, the roles of these SNPs in FM remain poorly understood. In addition, certain haplotypes have attracted the attention of investigators. Apart from individual SNPs, specific haplotypes of the COMT gene have been associated with pain sensitivity (individual responses to various stimuli) [59]. One particular COMT gene haplotype (ACCG), producing a defective enzyme, was associated with more sensitive perception of painful stimuli. Thus, the ACCG haplotype has been defined as a high-pain-sensitivity haplotype. Indeed, Vargas-Alarcon et al. [50] revealed associations among the ACCG haplotype, the presence of FM, and higher FIQ scores in FM patients.

In [206]:
## Migraine
mine = './23andme/genome_Adrien_Pinard_v4_Full_20180220145548.txt'
df_me = open_file(mine)
user_8 = './23andme/user8_file2_yearofbirth_unknown_sex_XX.23andme.txt'
df_8 = open_file(user_8)
user_495 = './23andme/user495_file223_yearofbirth_1965_sex_XX.23andme.txt'
df_495 = open_file(user_495)
user_637 = './23andme/user637_file293_yearofbirth_1989_sex_XY.23andme.txt'
df_637 = open_file(user_637)

In [96]:
## No migraine
user_1147 = './23andme/user1147_file598_yearofbirth_1948_sex_XY.23andme.txt'
df_1147 = open_file(user_1147)
user_1173 = './23andme/user1173_file609_yearofbirth_unknown_sex_XX.23andme.txt'
df_1173 = open_file(user_1173)

In [196]:
## Mix
user_60 = './23andme/user60_file27_yearofbirth_1982_sex_XY.23andme.txt'
df_60 = open_file(user_60)

In [197]:
df_8.shape, df_495.shape, df_60.shape

((966983, 4), (960566, 4), (991600, 4))

In [382]:
def check_genotype(df, rsid):
    res = df[df['rsid'] == rsid]['genotype']
    return rsid, res.iloc[0] if res.shape[0] > 0 else None
    
def check_COMT(df):
    return check_genotype(df, 'rs6269')

def check_COMT_2(df):
    return check_genotype(df, 'rs4818')

def check_COMT_61_P199P(df):
    return check_genotype(df, 'rs769224')

def check_COMT_H62H(df):
    return check_genotype(df, 'rs4633')

def check_COMT_V158M(df):
    return check_genotype(df, 'rs4680')

def check_MAO_A_R297R(df):
    return check_genotype(df, 'rs6323')

def check_MTHFR_A1298C(df):
    return check_genotype(df, 'rs1801131')

def check_MTHFR_C677T(df):
    return check_genotype(df, 'rs1801133')

check_COMT(df_8), check_COMT_H62H(df_8), check_COMT_2(df_8)

(('rs6269', 'AA'), ('rs4633', 'TT'), ('rs4818', None))

In [378]:
# migraine_users_id

In [200]:
df_60_me = pd.merge(df_me, df_60, how='inner', on=column_labels)

In [76]:
df_8_me = pd.merge(df_8, df_me, how='inner', on=column_labels)

In [82]:
df_8_me.shape

(138, 4)

In [86]:
df_8_me['chromosome'].unique()

array(['17', 'MT'], dtype=object)

In [201]:
df_60_8_me = pd.merge(df_8, df_60_me, how='inner', on=column_labels)

In [204]:
df_60_8_me.shape

(126, 4)

In [77]:
df_8_495_me = pd.merge(df_495, df_8_me, how='inner', on=column_labels)

In [81]:
df_8_495_me.shape

(124, 4)

In [83]:
df_8_495_me['chromosome'].unique()

array(['17', 'MT'], dtype=object)

In [90]:
df_8_495_me.groupby('chromosome')

In [379]:
# sns.pairplot(df_8_495_me)

In [98]:
## Asperger
df_1147_me = pd.merge(df_1147, df_me, how='inner', on=column_labels)

In [102]:
## ADHD
df_1173_me = pd.merge(df_1173, df_me, how='inner', on=column_labels)

In [104]:
df_1147_1173 = pd.merge(df_1147, df_1173, how='inner', on=column_labels)

In [209]:
df_1147_1173.shape

(531895, 4)

In [380]:
PREFIX_PATH = './23andme/'
onlyfiles = [f for f in listdir(PREFIX_PATH) if isfile(join(PREFIX_PATH, f))]

user_to_data_files = {}
for file in onlyfiles:
    key = file.split('_')[0].replace('user', '')
    user_to_data_files[key] = file
    
# user_to_data_files

In [375]:
df_model = df_model.assign(
    rs4633=None, 
    rs4680=None, 
    rs4818=None, 
    rs6269=None,
    rs6323=None,
    rs769224=None,
    rs1801131=None,
    rs1801133=None
)

In [377]:
# df_model

In [383]:
def populate_model_df_with_rsids():
    for user_id in migraine_users_id:
        filename = user_to_data_files.get(str(user_id), None)
    #     print(filename)
        if filename:
            current_df = open_file(PREFIX_PATH + filename)
    #         print(current_df.shape)
            rsid, value = check_COMT(current_df)
            df_model.loc[user_id, rsid] = value
            
            rsid, value = check_COMT_2(current_df)
            df_model.loc[user_id, rsid] = value
            
            rsid, value = check_COMT_61_P199P(current_df)
            df_model.loc[user_id, rsid] = value
            
            rsid, value = check_COMT_H62H(current_df)
            df_model.loc[user_id, rsid] = value
            
            rsid, value = check_COMT_V158M(current_df)
            df_model.loc[user_id, rsid] = value
            
            rsid, value = check_MAO_A_R297R(current_df)
            df_model.loc[user_id, rsid] = value
            
            rsid, value = check_MTHFR_A1298C(current_df)
            df_model.loc[user_id, rsid] = value
            
            rsid, value = check_MTHFR_C677T(current_df)
            df_model.loc[user_id, rsid] = value
                
populate_model_df_with_rsids()

In [401]:
df_model

,migraine,ADHD,IBS,HighBP,CFS,CF,rs4633,rs4680,rs4818,rs6269,rs6323,rs769224,rs1801131,rs1801133
id,,,,,,,,,,,,,,
144,0,0,0,0,0,0,None,None,None,None,None,None,None,None
139,0,0,0,0,0,0,None,None,None,None,None,None,None,None
124,0,1,0,0,0,0,None,None,None,None,None,None,None,None
126,0,0,0,0,0,0,None,None,None,None,None,None,None,None
141,0,0,0,0,0,0,None,None,None,None,None,None,None,None
137,0,0,0,0,0,0,None,None,None,None,None,None,None,None
125,0,0,0,0,0,0,None,None,None,None,None,None,None,None
143,0,0,0,0,0,0,None,None,None,None,None,None,None,None
123,0,0,0,0,0,0,None,None,None,None,None,None,None,None
